# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
raw_fold = 'D:/data/big2/BD1_4등급_데이터_점검1'

# Load

## 병합 result 최종 파일

In [4]:
# about 12s
name = '4등급_차량_리스트_양식_result'
file_name = f'{name}.csv'
result = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 28 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   차대번호    1502235 non-null  object 
 1   차량번호    1502235 non-null  object 
 2   제원관리번호  1502235 non-null  object 
 3   차종      1502235 non-null  object 
 4   용도      1502235 non-null  object 
 5   최초등록일   1502235 non-null  int64  
 6   연식      1502235 non-null  int64  
 7   제작일자    1335178 non-null  float64
 8   검사유효일   1502223 non-null  float64
 9   인증번호    1382225 non-null  object 
 10  등급      1502235 non-null  int64  
 11  차명      1502235 non-null  object 
 12  차종분류    1502235 non-null  object 
 13  차종유형    1502235 non-null  object 
 14  자동차형식   1501958 non-null  object 
 15  제작사명    1492590 non-null  object 
 16  연료      1502235 non-null  object 
 17  엔진형식    1500400 non-null  object 
 18  총중량     1502234 non-null  float64
 19  적재중량    1501728 non-null  float64
 20  엔진출력    1501882 non-null

# 전처리

## 휘발유 차량만

In [5]:
result['연료'].value_counts(dropna=False)

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [7]:
result1 = result.loc[result['연료'] == '휘발유'].reset_index(drop=True)
result1.shape

(322983, 28)

## 4등급 차량만

In [9]:
result1['등급'].value_counts(dropna=False)

4    322983
Name: 등급, dtype: int64

In [11]:
result1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

# 분석

In [13]:
grade_list = []
for f, y, cy in tqdm(result1[['연료', '제작일자', '연식']].values):
    if (f == '휘발유') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 322983/322983 [00:00<00:00, 562057.93it/s]


322983

In [14]:
grade_list[:5]

['X', 'A', 'B', 'A', 'B']

In [15]:
num = 85420
grade_list[num:num+10]

['B', 'B', 'B', 'B', 'B', 'B', 'X', 'B', 'B', 'B']

In [16]:
grade_list[-5:]

['B', 'B', 'B', 'A', 'B']

In [17]:
df = result1.copy()

In [18]:
df['Grade'] = grade_list

In [19]:
num_by_fuel = df.groupby(['연료'])['차대번호'].count().reset_index().sort_values('차대번호', ascending=False)
num_by_fuel = num_by_fuel.rename(columns={'차대번호':'대수'})
num_by_fuel

,연료,대수
0,휘발유,322983


In [20]:
num_by_grade = df.groupby(['연료', 'Grade'])['차대번호'].count()
num_by_grade = num_by_grade.rename('대수')
num_by_grade

연료   Grade
휘발유  A         28923
     B        280071
     X         13989
Name: 대수, dtype: int64

In [21]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'Grade'],
      dtype='object')

In [23]:
today_date = datetime.today().strftime("%Y.%m.%d")
today_date

'2023.03.24'

In [24]:
# about 5s
df.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 휘발유 Grade 추가({today_date}).csv'), index=False, encoding='cp949')

## [출력] EG 통계

In [25]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.03.24_PM.16.36.00'

In [26]:
with pd.ExcelWriter(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 휘발유 EG 통계 {today_date}.xlsx')) as writer:
    num_by_fuel.to_excel(writer, sheet_name='연료별', index=False)
    num_by_grade.to_excel(writer, sheet_name='등급별')

# 코드 마지막